In [1]:
import main, json, importlib, time
import numpy as np
np.set_printoptions(suppress = True)
import qiskit as qk
from qiskit import Aer
from qiskit.tools.visualization import plot_histogram
import matplotlib.pyplot as plt
from qiskit.visualization import plot_state_city, plot_bloch_multivector
from qiskit.visualization import plot_state_paulivec, plot_state_hinton
from qiskit.visualization import plot_state_qsphere
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error


In [2]:
def CRnU(circuit, control, target, ancilla):
    #match shape to the one from the paper
    # circuit.swap(control, target)

    circuit.cnot(ancilla, target)
    circuit.cnot(control, target)
    circuit.h(ancilla)
    circuit.t(control)
    circuit.tdg(target)
    circuit.t(ancilla)
    circuit.cnot(ancilla, target)
    circuit.cnot(control, ancilla)
    circuit.t(target)
    circuit.cnot(control, target)
    circuit.tdg(ancilla)
    circuit.tdg(target)
    circuit.cnot(control, ancilla)
    circuit.cnot(ancilla, target)
    circuit.t(target)
    circuit.h(ancilla)
    circuit.cnot(ancilla, target)

    #swap back
    # circuit.swap(control, target)

def ZeroAncCRn(circuit, control, target, theta):
    # not working so far
    circuit.p(2*theta, control)
    circuit.p(2*theta, target)
    circuit.cnot(control, target)
    circuit.p(-2*theta, target)
    circuit.cnot(control, target)

In [3]:
def CRnGate(circuit, control, target, ancilla, theta):
    CRnU(circuit, control, target, ancilla)
    circuit.p(theta, ancilla)
    CRnU(circuit, control, target, ancilla)